In [1]:
import pandas as pd
import ast

In [2]:
path_sankoff = "../results/sankoff_labeling_analysis/equal/FA_sankoff_equal.csv"
data_san = pd.read_csv(path_sankoff)
print(data_san.head())

  characters  fa_length                                            fa_list
0     K02257          3  ['Actinomycetes', 'Pseudomonadota', 'Staphyloc...
1     K02274          2                ['Actinomycetes', 'Pseudomonadota']
2     K02259          3  ['Actinomycetes', 'Pseudomonadota', 'Staphyloc...
3     K02275          2                ['Actinomycetes', 'Pseudomonadota']
4     K01772          7  ['Actinomycetota', 'Campylobacterales', 'Pseud...


In [3]:
#Highways that are unique to the Sankoff Labeling
san_highways =[('Actinomycetes', 'T06554'), ('Micrococcaceae', 'Pseudomonadota'), ('Actinomycetes', 'T04453'), ('Pseudomonadota', 'T06554'), ('Actinomycetes', 'T08201'), ('Actinomycetes', 'T06794'), ('Actinomycetes', 'Staphylococcus')]

#convert the string column into a list
data_san['fa_list'] = data_san['fa_list'].apply(ast.literal_eval)

#Look for characters for the highway dict
highway_dict = dict()
for a,b in san_highways:
    temp = [a,b]
    temp = sorted(temp)
    key = '-'.join(temp)
    chars_in_highway = []
    characters = list(data_san['characters'])
    fa_list = list(data_san['fa_list'])
    for i in range(0,len(characters)):
        char_list = set(fa_list[i])
        if a in char_list:
            if b in char_list:
                chars_in_highway.append(characters[i])
    highway_dict[key] = list(chars_in_highway)

#Print Results
for key in highway_dict:
    print("\t",key,"\t",highway_dict[key])

	 Actinomycetes-T06554 	 ['K01011', 'K00135', 'K01589', 'K03820', 'K00252', 'K00459', 'K14162', 'K00641', 'K01692', 'K03652', 'K01679', 'K00140']
	 Micrococcaceae-Pseudomonadota 	 ['K01669', 'K06999', 'K06204', 'K00362', 'K09791', 'K00390', 'K01029', 'K01595', 'K03117', 'K00363', 'K09773', 'K02303', 'K00019', 'K10764', 'K10804']
	 Actinomycetes-T04453 	 ['K01011', 'K00681', 'K05786', 'K00390', 'K02446', 'K00641', 'K00140']
	 Pseudomonadota-T06554 	 ['K01669', 'K01011', 'K00135', 'K01589', 'K00252', 'K03782', 'K01029', 'K14162', 'K08973', 'K00019', 'K11473', 'K01679', 'K10764', 'K00140', 'K00208']
	 Actinomycetes-T08201 	 ['K00242', 'K05786', 'K09791', 'K06916', 'K00241', 'K00641', 'K03652', 'K07399']
	 Actinomycetes-T06794 	 ['K00632', 'K01903', 'K00681', 'K08296', 'K01902', 'K00242', 'K00252', 'K00121', 'K06916', 'K00140']
	 Actinomycetes-Staphylococcus 	 ['K02257', 'K02259', 'K00627', 'K03635', 'K01255', 'K03636', 'K06886', 'K01903', 'K00681', 'K01880', 'K01902', 'K05786', 'K00459', 

A striking difference with Genesis Algorithm are the highways that connect two comparable nodes:
- Pseudomonadota & T06554
- Actinomycetes & T06794

In [4]:
highway_1 = list(highway_dict["Pseudomonadota-T06554"])

print("There are ",len(highway_1)," characters 'Pseudomonadota-T06554' highway")

highway_2 = list(highway_dict["Actinomycetes-T06794"])

print("There are ",len(highway_2)," characters 'Actinomycetes-T06794' highway")

There are  15  characters 'Pseudomonadota-T06554' highway
There are  10  characters 'Actinomycetes-T06794' highway


In [5]:
from treebased import TB_Network
import matrix as mx
import pickle as pkl
import tools as ts
import pandas as pd

## Differences between Sankoff and Genesis with the characters involved
We will now calculate the differences between both methods using only the characters that were present in the highways with regains.

In [7]:
#Import species tree data
data = pd.read_csv("../data/regain_analysis/interphylum_species_50.csv")
data['name_on_ncbi_tree'] = data['name_on_ncbi_tree'].str.replace("'","")
print (data.head())

#Create a dated species tree
file = open("../data/regain_analysis/species_tree_ultra.pkl","rb")
S = pkl.load(file)
print(S.to_newick())

                ID  ncbiid                       species kegg_id  \
0          74426.4   74426       Collinsella aerofaciens  T05143   
1         679935.3  679935          Alistipes finegoldii  T02133   
2          88431.3   88431             Dorea longicatena  T08069   
3  GCA_003469045.1   39486         Dorea formicigenerans  T08666   
4        226186.12  226186  Bacteroides thetaiotaomicron  T00122   

           phylum                      name_on_ncbi_tree  
0  Actinobacteria                Collinsella aerofaciens  
1   Bacteroidetes         Alistipes finegoldii DSM 17242  
2      Firmicutes                      Dorea longicatena  
3      Firmicutes                  Dorea formicigenerans  
4   Bacteroidetes  Bacteroides thetaiotaomicron VPI-5482  
(T08201:1.0,T04453:1.0,T01461:1.0,(T08780:0.8056460768611685,(T04021:0.25795113603999975,T00577:0.25795113603999975)Campylobacter:0.5476949408211688)Campylobacterales:0.19435392313883149,(T06085:0.5149615673651129,((T04721:0.475254243154

In [7]:
#Create character-state matrix
m_highway_1 = mx.generate_from_KEGG_v1(list(data['kegg_id']),highway_1)
file1 = open('../results/regain_analysis/csmatrix_hw1.pkl', 'wb')
pkl.dump(m_highway_1,file1)
file1.close()


	 Building char-state matrix: 
|████████████████████████████████████████| 45/45 [100%] in 14:41.8 (0.05/s) 


In [17]:

#Create a TB Network
N = TB_Network(S.root)
N.init_base_from_tralda(S,len(highway_1))

for leaf in N.leaves():
    leaf.chars = m_highway_1[leaf.label]

ts.print_cs_matrix(N)


	 Character-state Matrix
	 taxa 		 characters
	  T08201 	 [False, False, False, False, False, False, False, False, False, False, False, False, False, False, True]
	  T04453 	 [False, True, False, False, False, False, False, False, False, False, False, False, False, True, False]
	  T01461 	 [False, False, False, False, False, False, False, False, False, False, False, True, False, False, False]
	  T08780 	 [False, False, True, False, False, False, True, False, True, True, False, True, False, False, False]
	  T04021 	 [False, False, False, False, False, False, False, False, False, False, True, False, False, False, True]
	  T00577 	 [False, False, True, False, False, False, False, False, True, False, True, False, False, False, True]
	  T06085 	 [True, True, True, True, True, True, True, True, True, True, True, True, True, True, True]
	  T04721 	 [True, True, True, True, True, True, True, True, True, True, True, True, False, True, True]
	  T04481 	 [True, True, True, True, True, True, True,

In [18]:
N.sankoff_labeling(1.0,1.0)

#Calculate first appearance nodes
fas = N.get_fas_by_state_change()

print("Label ","\tcharacters")
for n in N.postorder():
    print(n.label," ",n.chars)

for f in fas:
    fa_list = [node.label for node in fas[f]]
    print (highway_1[f], "\t",fa_list)


file2 = open('../results/regain_analysis/sankoff_network_hw1.pkl', 'wb')
pkl.dump(N,file2)
file1.close()

Label  	characters
T08201   [False, False, False, False, False, False, False, False, False, False, False, False, False, False, True]
T04453   [False, True, False, False, False, False, False, False, False, False, False, False, False, True, False]
T01461   [False, False, False, False, False, False, False, False, False, False, False, True, False, False, False]
T08780   [False, False, True, False, False, False, True, False, True, True, False, True, False, False, False]
T04021   [False, False, False, False, False, False, False, False, False, False, True, False, False, False, True]
T00577   [False, False, True, False, False, False, False, False, True, False, True, False, False, False, True]
Campylobacter   [False, False, False, False, False, False, False, False, False, False, True, False, False, False, True]
Campylobacterales   [False, False, True, False, False, False, False, False, True, False, False, False, False, False, False]
T06085   [True, True, True, True, True, True, True, True, True

In [13]:
def print_newick(node, i):
    """
    Creates a Newick string of the subtree rooted at a given node.
    The final string for the entire tree will end with a semicolon.

    Args:
        node (TB_Node): The root of the subtree to be printed.
        i (int): The index of the character to check in node.chars.

    Returns:
        str: A Newick string representation of the subtree.
    """
    if not node:
        return ""

    # Base case: a leaf node
    if not node.children:
        label = node.label
        if node.chars[i]:
            label += "<*>"
        return label

    # Recursive step: an internal node
    subtree_strings = []
    for child in node.children:
        subtree_strings.append(print_newick(child, i))

    newick_str = ",".join(subtree_strings)
    
    label = node.label if node.label else ""
    if node.chars[i]:
        label += "<*>"

    # Add the semicolon only at the very end of the root node's string
    if not node.parent:  # Check if this is the root of the entire tree
        return f"({newick_str}){label};"
    else:
        return f"({newick_str}){label}"

print("\t For Sankoff Labeling: ")
print("\t KO \t Newick String")
for char in range(0,len(highway_1)):
    print("\t",highway_1[char],"\t",print_newick(N.root, char))
    

	 For Sankoff Labeling: 
	 KO 	 Newick String
	 K01669 	 (T08201,T04453,T01461,(T08780,(T04021,T00577)Campylobacter)Campylobacterales,(T06085<*>,((T04721<*>,T04481<*>)Comamonadaceae<*>,(T00925<*>,T00398<*>,T05738<*>)Burkholderiaceae<*>)Burkholderiales<*>,(T03770<*>,T06554<*>)Gammaproteobacteria<*>)Pseudomonadota<*>,(T00964,T06584,(T04182<*>,(T05095,T07943,T06719,(T08069,T08666)Dorea)Lachnospiraceae)Eubacteriales,((T01954,T04115)Lactobacillus,(T03970,T06252)Staphylococcus)Bacilli)Bacillota,((T00984,T05143)Coriobacteriia,((T03496,T02505,T04222,T01842)Bifidobacterium,(T05843<*>,(T05201<*>,T00701,T00447<*>)Micrococcaceae<*>)Micrococcales<*>,T02545<*>,((T07267,T07354)Corynebacterium,T06794<*>)Mycobacteriales)Actinomycetes<*>)Actinomycetota,(T01485,T02133,(T07941,T00122,T06523)Bacteroides)Bacteroidales)Bacteria;
	 K01011 	 (T08201,T04453,T01461,(T08780,(T04021<*>,T00577<*>)Campylobacter<*>)Campylobacterales,(T06085<*>,((T04721<*>,T04481<*>)Comamonadaceae<*>,(T00925<*>,T00398<*>,T05738<*>)Bur

IndexError: list index out of range

Introducing the genesis labeling function. Will add this function as a method for the TB Nework later :)

In [16]:
def genesis_labeling(TB,loss_cost,fa_cost):
    def _base_case_value(dp_table,v,label,gain,char):
        if v.chars[char] == False:
            if label == 0 and gain == 0:
                return(0)
            else:
                return(1000000000) #not inf but something very big
        else:
            if label == 1:
                return(0)
            else:
                return(1000000000) #same

    def _inner_case_value(dp_table,v,current_label,current_gain,M):
        #Special case: V[v,0,1]
        # This case fixes one child as origin and leaves the rest to fate
        if current_label == 0 and current_gain == 1:
            child_sum = []
            for child in v.children:
                origin_penalty = []
                for l in [0,1]:
                    #origin_penalty.append(dp_table[child][l,1] + M[current_label,l])
                    origin_penalty.append(dp_table[child][l,1])
                brother_sum = 0
                for brother in v.children:
                    if brother != child:
                        label_cost = []
                        for l in [0,1]:
                            label_cost.append(dp_table[brother][l,0] + M[current_label,l])
                        brother_sum = brother_sum + min(label_cost)
                child_sum.append(min(origin_penalty) + brother_sum)
            return(min(child_sum))
        else:
            cost = 0
            for child in v.children:
                label_costs = []
                for l in [1,0]:
                    label_costs.append(dp_table[child][l,0] + M[current_label,l])
                cost = cost + min(label_costs)
            return cost

    #Backtracking
    def _find_best_child_origin(dp_table,v,M):
        argmin = []
        children = list(v.children)
        for child in v.children:
            origin_penalty = []
            for l in [0,1]:
                origin_penalty.append(dp_table[child][l,1] + M[0,l])
            brother_sum = 0
            for brother in v.children:
                if brother != child:
                    label_cost = []
                    for l in [0,1]:
                        label_cost.append(dp_table[brother][l,0] + M[0,l])
                    brother_sum = brother_sum + min(label_cost)
            argmin.append(min(origin_penalty) + brother_sum)
        
        index_min = np.argmin(argmin)
        return(children[index_min])


    #cost matrix
    M = np.array([[0.0,fa_cost],[loss_cost,0.0]])
    dp_table = dict()

    # Main algo
    for char in range(0,len(TB.root.chars)):
        dp_table = dict()

        #main algo
        for v in TB.postorder():

            dp_table[v] = dict()
            for label in [0,1]:
                for gain in [0,1]:
                    if v.is_leaf():
                        dp_table[v][label,gain] = _base_case_value(dp_table,v,label,gain,char)
                    else:
                        dp_table[v][label,gain] = _inner_case_value(dp_table,v,label,gain,M)

        #backtracking
        origin = None
        for v in TB.preorder():
            if v == N.root:
                if dp_table[v][1,1] <= dp_table[v][0,1]:
                    v.chars[char] = True
                    origin = N.root
                else:
                    v.chars[char] = False
                    #choose the origin
                    origin = _find_best_child_origin(dp_table,v,M)
           
            else:
                if v == origin:
                    if dp_table[v][1,1] <= dp_table[v][0,1]:
                        v.chars[char] = True
                    else:
                        v.chars[char] = False
                        #choose the origin
                        if not v.is_leaf():
                            origin = _find_best_child_origin(dp_table,v,M)
                else:
                    if dp_table[v][1,0] <= dp_table[v][0,0]:
                        v.chars[char] = True
                    else:
                        v.chars[char] = False


In [18]:
import numpy as np

#Create another TB Network
N_genesis = TB_Network(S.root)
N_genesis.init_base_from_tralda(S,len(highway_1))

for leaf in N_genesis.leaves():
    leaf.chars = m_highway_1[leaf.label]

ts.print_cs_matrix(N_genesis)

genesis_labeling(N_genesis, 1.0,1.0)

print("\t For Genesis Labeling: ")
print("\t KO \t Newick String")
for char in range(0,len(highway_1)):
    print("\t",highway_1[char],"\t",print_newick(N_genesis.root, char))



file3 = open('../results/regain_analysis/genesis_network_hw1.pkl', 'wb')
pkl.dump(N_genesis,file3)
file3.close()

NameError: name 'm_highway_1' is not defined

In [8]:
#Create character-state matrix
m_highway_2 = mx.generate_from_KEGG_v1(list(data['kegg_id']),highway_2)
file2 = open('../results/regain_analysis/csmatrix_hw1.pkl', 'wb')
pkl.dump(m_highway_2,file2)
file2.close()

	 Building char-state matrix: 
|████████████████████████████████████████| 45/45 [100%] in 8:48.3 (0.09/s) 


Now we do the same with the characters from the other popular re-gain highway

In [9]:
#Create a TB Network
N = TB_Network(S.root)
N.init_base_from_tralda(S,len(highway_2))

for leaf in N.leaves():
    leaf.chars = m_highway_2[leaf.label]

ts.print_cs_matrix(N)

	 Character-state Matrix
	 taxa 		 characters
	  T08201 	 [0, 0, 0, 0, 0, 1, 0, 0, 1, 0]
	  T04453 	 [0, 0, 1, 0, 0, 0, 0, 0, 0, 1]
	  T01461 	 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
	  T08780 	 [0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
	  T04021 	 [0, 1, 0, 1, 1, 0, 0, 0, 0, 0]
	  T00577 	 [0, 1, 0, 1, 1, 0, 0, 0, 0, 0]
	  T06085 	 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
	  T04721 	 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
	  T04481 	 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
	  T00925 	 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
	  T00398 	 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
	  T05738 	 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
	  T03770 	 [1, 1, 1, 1, 1, 1, 0, 1, 1, 0]
	  T06554 	 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
	  T00964 	 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
	  T06584 	 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
	  T04182 	 [1, 0, 1, 0, 0, 0, 0, 0, 0, 0]
	  T05095 	 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
	  T07943 	 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
	  T06719 	 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
	  T08069 	 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
	  T08666 	 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
	  T0195

In [11]:
N.sankoff_labeling(1.0,1.0)

#Calculate first appearance nodes
fas = N.get_fas_by_state_change()

print("Label ","\tcharacters")
for n in N.postorder():
    print(n.label," ",n.chars)

for f in fas:
    fa_list = [node.label for node in fas[f]]
    print (highway_2[f], "\t",fa_list)


file2 = open('../results/regain_analysis/sankoff_network_hw2.pkl', 'wb')
pkl.dump(N,file2)
file2.close()

Label  	characters
T08201   [False, False, False, False, False, True, False, False, True, False]
T04453   [False, False, True, False, False, False, False, False, False, True]
T01461   [False, False, False, False, False, False, False, False, False, False]
T08780   [False, False, True, False, False, False, False, False, False, False]
T04021   [False, True, False, True, True, False, False, False, False, False]
T00577   [False, True, False, True, True, False, False, False, False, False]
Campylobacter   [False, True, False, True, True, False, False, False, False, False]
Campylobacterales   [False, False, False, False, False, False, False, False, False, False]
T06085   [True, True, True, True, True, True, True, True, True, True]
T04721   [True, True, True, True, True, True, True, True, True, True]
T04481   [True, True, True, True, True, True, True, True, True, True]
Comamonadaceae   [True, True, True, True, True, True, True, True, True, True]
T00925   [True, True, True, True, True, True, Tru

In [14]:
print("\t For Sankoff Labeling: ")
print("\t KO \t Newick String")
for char in range(0,len(highway_2)):
    print("\t",highway_2[char],"\t",print_newick(N.root, char))

	 For Sankoff Labeling: 
	 KO 	 Newick String
	 K00632 	 (T08201,T04453,T01461,(T08780,(T04021,T00577)Campylobacter)Campylobacterales,(T06085<*>,((T04721<*>,T04481<*>)Comamonadaceae<*>,(T00925<*>,T00398<*>,T05738<*>)Burkholderiaceae<*>)Burkholderiales<*>,(T03770<*>,T06554<*>)Gammaproteobacteria<*>)Pseudomonadota<*>,(T00964,T06584,(T04182<*>,(T05095,T07943,T06719,(T08069,T08666)Dorea)Lachnospiraceae)Eubacteriales,((T01954,T04115)Lactobacillus,(T03970,T06252)Staphylococcus)Bacilli)Bacillota,((T00984,T05143)Coriobacteriia,((T03496,T02505,T04222,T01842)Bifidobacterium,(T05843<*>,(T05201<*>,T00701,T00447<*>)Micrococcaceae<*>)Micrococcales<*>,T02545<*>,((T07267,T07354)Corynebacterium,T06794<*>)Mycobacteriales)Actinomycetes<*>)Actinomycetota,(T01485,T02133,(T07941,T00122,T06523)Bacteroides)Bacteroidales)Bacteria;
	 K01903 	 (T08201,T04453,T01461,(T08780,(T04021<*>,T00577<*>)Campylobacter<*>)Campylobacterales,(T06085<*>,((T04721<*>,T04481<*>)Comamonadaceae<*>,(T00925<*>,T00398<*>,T05738<*>)Bur

In [19]:
#Create another TB Network
N_genesis = TB_Network(S.root)
N_genesis.init_base_from_tralda(S,len(highway_2))

for leaf in N_genesis.leaves():
    leaf.chars = m_highway_2[leaf.label]

ts.print_cs_matrix(N_genesis)

genesis_labeling(N_genesis, 1.0,1.0)

print("\t For Genesis Labeling: ")
print("\t KO \t Newick String")
for char in range(0,len(highway_2)):
    print("\t",highway_2[char],"\t",print_newick(N_genesis.root, char))



file3 = open('../results/regain_analysis/genesis_network_hw2.pkl', 'wb')
pkl.dump(N_genesis,file3)
file3.close()

	 Character-state Matrix
	 taxa 		 characters
	  T08201 	 [False, False, False, False, False, True, False, False, True, False]
	  T04453 	 [False, False, True, False, False, False, False, False, False, True]
	  T01461 	 [False, False, False, False, False, False, False, False, False, False]
	  T08780 	 [False, False, True, False, False, False, False, False, False, False]
	  T04021 	 [False, True, False, True, True, False, False, False, False, False]
	  T00577 	 [False, True, False, True, True, False, False, False, False, False]
	  T06085 	 [True, True, True, True, True, True, True, True, True, True]
	  T04721 	 [True, True, True, True, True, True, True, True, True, True]
	  T04481 	 [True, True, True, True, True, True, True, True, True, True]
	  T00925 	 [True, True, True, True, True, True, True, True, True, True]
	  T00398 	 [True, True, True, True, True, True, True, True, True, True]
	  T05738 	 [True, True, True, True, True, True, True, True, True, True]
	  T03770 	 [True, True, True